In [ ]:
!pip install seaborn catboost --quiet

import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn
from keras.models import Sequential, load_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler,OneHotEncoder
from sklearn.metrics import confusion_matrix
from tensorflow.keras.utils import to_categorical

In [ ]:
import os
import pandas as pd
import kagglehub

# Download the 2018 CICIDS dataset
path_2018 = kagglehub.dataset_download("solarmainframe/ids-intrusion-csv")
print("Dataset folder for 2018:", path_2018)

# List all CSV files in the 2018 dataset directory
csv_files_2018 = [f for f in os.listdir(path_2018) if f.endswith('.csv')]
print(f"Found {len(csv_files_2018)} CSV files in 2018 dataset.")

# Load only first 5 CSV files for faster processing
csv_files_2018_subset = csv_files_2018[:5]

dfs_2018 = []
for file_name in csv_files_2018_subset:
    file_path = os.path.join(path_2018, file_name)
    print("Loading:", file_path)
    df = pd.read_csv(file_path, low_memory=False)
    dfs_2018.append(df)

# Combine loaded CSV files into one DataFrame
df = pd.concat(dfs_2018, ignore_index=True)
print("Shape of combined 2018 data:", df.shape)

# Preview the combined data
df.head()


,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,112641719,3,0,0,0,0,0,0.000000,0.000000,...,0,0.0,0.0,0,0,56320859.5,139.300036,56320958,56320761,Benign
1,0,112641466,3,0,0,0,0,0,0.000000,0.000000,...,0,0.0,0.0,0,0,56320733.0,114.551299,56320814,56320652,Benign
2,0,112638623,3,0,0,0,0,0,0.000000,0.000000,...,0,0.0,0.0,0,0,56319311.5,301.934596,56319525,56319098,Benign
3,6,6453966,15,10,1239,2273,744,0,82.600000,196.741237,...,32,0.0,0.0,0,0,0.0,0.000000,0,0,Benign
4,6,8804066,14,11,1143,2209,744,0,81.642857,203.745545,...,32,0.0,0.0,0,0,0.0,0.000000,0,0,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2798299,6,19276110,5,10,286,738,103,0,57.200000,52.485236,...,20,9667037.0,0.0,9667037,9667037,9609073.0,0.000000,9609073,9609073,Benign
2798300,6,85992425,2,0,0,0,0,0,0.000000,0.000000,...,20,0.0,0.0,0,0,85992425.0,0.000000,85992425,85992425,Benign
2798301,17,79134,1,1,41,98,41,41,41.000000,0.000000,...,8,0.0,0.0,0,0,0.0,0.000000,0,0,Benign
2798302,6,10053785,11,12,1321,1659,677,0,120.090909,197.116440,...,20,0.0,0.0,0,0,0.0,0.000000,0,0,Benign


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2798304 entries, 0 to 2798303
Data columns (total 70 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Protocol           int64  
 1   Flow Duration      int64  
 2   Tot Fwd Pkts       int64  
 3   Tot Bwd Pkts       int64  
 4   TotLen Fwd Pkts    int64  
 5   TotLen Bwd Pkts    int64  
 6   Fwd Pkt Len Max    int64  
 7   Fwd Pkt Len Min    int64  
 8   Fwd Pkt Len Mean   float64
 9   Fwd Pkt Len Std    float64
 10  Bwd Pkt Len Max    int64  
 11  Bwd Pkt Len Min    int64  
 12  Bwd Pkt Len Mean   float64
 13  Bwd Pkt Len Std    float64
 14  Flow Byts/s        float64
 15  Flow Pkts/s        float64
 16  Flow IAT Mean      float64
 17  Flow IAT Std       float64
 18  Flow IAT Max       int64  
 19  Flow IAT Min       int64  
 20  Fwd IAT Tot        int64  
 21  Fwd IAT Mean       float64
 22  Fwd IAT Std        float64
 23  Fwd IAT Max        int64  
 24  Fwd IAT Min        int64  
 25  Bwd IAT Tot       

In [ ]:
df.columns

Index(['Protocol', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts',
       'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd URG Flags', 'Fwd Header Len',
       'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min',
       'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var',
       'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt',
       'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt',
       'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg',
       'Subflow Fwd Pkts'

In [ ]:
df['Label'].unique()

array(['Benign', 'FTP-BruteForce', 'SSH-Bruteforce', 'Infilteration',
       'Brute Force -Web', 'Brute Force -XSS', 'SQL Injection'],
      dtype=object)

In [ ]:
df['Label'].value_counts()

Benign              2589517
Infilteration        113819
SSH-Bruteforce        94048
Brute Force -Web        555
Brute Force -XSS        228
SQL Injection            84
FTP-BruteForce           53
Name: Label, dtype: int64

In [ ]:
Label = pd.DataFrame(df.Label)
Label

,Label
0,Benign
1,Benign
2,Benign
3,Benign
4,Benign
...,...
2798299,Benign
2798300,Benign
2798301,Benign
2798302,Benign


In [ ]:
std_scaler = MinMaxScaler()
def standardization(df,col):
    for i in col:
        arr = df[i]
        arr = np.array(arr)
        df[i] = std_scaler.fit_transform(arr.reshape(len(arr),1))
    return df

numeric_col = df.select_dtypes(include='number').columns
df = standardization(df,numeric_col)

<IPython.core.display.Javascript object>

In [ ]:
df

,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0.000000,0.938681,0.000016,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.469340,0.000002,0.469341,0.469340,Benign
1,0.000000,0.938679,0.000016,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.469339,0.000002,0.469340,0.469339,Benign
2,0.000000,0.938655,0.000016,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.469328,0.000004,0.469329,0.469326,Benign
3,0.352941,0.053783,0.000115,0.000081,8.580817e-06,1.453693e-05,0.011546,0.000000,0.007364,0.012483,...,0.666667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Benign
4,0.352941,0.073367,0.000107,0.000089,7.915959e-06,1.412762e-05,0.011546,0.000000,0.007278,0.012927,...,0.666667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2798299,0.352941,0.160634,0.000033,0.000081,1.980721e-06,4.719864e-06,0.001598,0.000000,0.005099,0.003330,...,0.416667,0.084799,0.0,0.084799,0.084799,0.080076,0.000000,0.080076,0.080076,Benign
2798300,0.352941,0.716604,0.000008,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.416667,0.000000,0.0,0.000000,0.000000,0.716604,0.000000,0.716604,0.716604,Benign
2798301,1.000000,0.000659,0.000000,0.000008,2.839496e-07,6.267571e-07,0.000636,0.028082,0.003655,0.000000,...,0.166667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Benign
2798302,0.352941,0.083782,0.000082,0.000097,9.148716e-06,1.061010e-05,0.010506,0.000000,0.010706,0.012507,...,0.416667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Benign


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2798304 entries, 0 to 2798303
Data columns (total 70 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Protocol           float64
 1   Flow Duration      float64
 2   Tot Fwd Pkts       float64
 3   Tot Bwd Pkts       float64
 4   TotLen Fwd Pkts    float64
 5   TotLen Bwd Pkts    float64
 6   Fwd Pkt Len Max    float64
 7   Fwd Pkt Len Min    float64
 8   Fwd Pkt Len Mean   float64
 9   Fwd Pkt Len Std    float64
 10  Bwd Pkt Len Max    float64
 11  Bwd Pkt Len Min    float64
 12  Bwd Pkt Len Mean   float64
 13  Bwd Pkt Len Std    float64
 14  Flow Byts/s        float64
 15  Flow Pkts/s        float64
 16  Flow IAT Mean      float64
 17  Flow IAT Std       float64
 18  Flow IAT Max       float64
 19  Flow IAT Min       float64
 20  Fwd IAT Tot        float64
 21  Fwd IAT Mean       float64
 22  Fwd IAT Std        float64
 23  Fwd IAT Max        float64
 24  Fwd IAT Min        float64
 25  Bwd IAT Tot       

In [ ]:
X = df.drop('Label', axis=1)
y = df['Label']

In [ ]:
encoder = OneHotEncoder(sparse=False)
y_encoded = encoder.fit_transform(y.values.reshape(-1, 1))
y_encoded = pd.DataFrame(y_encoded)
y_encoded

C:\Users\aryan\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,0,1,2,3,4,5,6
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
2798299,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2798300,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2798301,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2798302,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X

,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,0.000000,0.938681,0.000016,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.469340,0.000002,0.469341,0.469340
1,0.000000,0.938679,0.000016,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.469339,0.000002,0.469340,0.469339
2,0.000000,0.938655,0.000016,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.469328,0.000004,0.469329,0.469326
3,0.352941,0.053783,0.000115,0.000081,8.580817e-06,1.453693e-05,0.011546,0.000000,0.007364,0.012483,...,0.000049,0.666667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.352941,0.073367,0.000107,0.000089,7.915959e-06,1.412762e-05,0.011546,0.000000,0.007278,0.012927,...,0.000049,0.666667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2798299,0.352941,0.160634,0.000033,0.000081,1.980721e-06,4.719864e-06,0.001598,0.000000,0.005099,0.003330,...,0.000025,0.416667,0.084799,0.0,0.084799,0.084799,0.080076,0.000000,0.080076,0.080076
2798300,0.352941,0.716604,0.000008,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.416667,0.000000,0.0,0.000000,0.000000,0.716604,0.000000,0.716604,0.716604
2798301,1.000000,0.000659,0.000000,0.000008,2.839496e-07,6.267571e-07,0.000636,0.028082,0.003655,0.000000,...,0.000000,0.166667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2798302,0.352941,0.083782,0.000082,0.000097,9.148716e-06,1.061010e-05,0.010506,0.000000,0.010706,0.012507,...,0.000058,0.416667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
df = pd.concat([X, y_encoded], axis=1)
df

,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,...,Idle Std,Idle Max,Idle Min,0,1,2,3,4,5,6
0,0.000000,0.938681,0.000016,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000002,0.469341,0.469340,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.938679,0.000016,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000002,0.469340,0.469339,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.938655,0.000016,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000004,0.469329,0.469326,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.352941,0.053783,0.000115,0.000081,8.580817e-06,1.453693e-05,0.011546,0.000000,0.007364,0.012483,...,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.352941,0.073367,0.000107,0.000089,7.915959e-06,1.412762e-05,0.011546,0.000000,0.007278,0.012927,...,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2798299,0.352941,0.160634,0.000033,0.000081,1.980721e-06,4.719864e-06,0.001598,0.000000,0.005099,0.003330,...,0.000000,0.080076,0.080076,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2798300,0.352941,0.716604,0.000008,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.716604,0.716604,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2798301,1.000000,0.000659,0.000000,0.000008,2.839496e-07,6.267571e-07,0.000636,0.028082,0.003655,0.000000,...,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2798302,0.352941,0.083782,0.000082,0.000097,9.148716e-06,1.061010e-05,0.010506,0.000000,0.010706,0.012507,...,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2798304 entries, 0 to 2798303
Data columns (total 76 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Protocol           float64
 1   Flow Duration      float64
 2   Tot Fwd Pkts       float64
 3   Tot Bwd Pkts       float64
 4   TotLen Fwd Pkts    float64
 5   TotLen Bwd Pkts    float64
 6   Fwd Pkt Len Max    float64
 7   Fwd Pkt Len Min    float64
 8   Fwd Pkt Len Mean   float64
 9   Fwd Pkt Len Std    float64
 10  Bwd Pkt Len Max    float64
 11  Bwd Pkt Len Min    float64
 12  Bwd Pkt Len Mean   float64
 13  Bwd Pkt Len Std    float64
 14  Flow Byts/s        float64
 15  Flow Pkts/s        float64
 16  Flow IAT Mean      float64
 17  Flow IAT Std       float64
 18  Flow IAT Max       float64
 19  Flow IAT Min       float64
 20  Fwd IAT Tot        float64
 21  Fwd IAT Mean       float64
 22  Fwd IAT Std        float64
 23  Fwd IAT Max        float64
 24  Fwd IAT Min        float64
 25  Bwd IAT Tot       

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y_encoded,test_size=0.2)
print(X_train.shape," ",X_test.shape)
print(y_train.shape," ",y_test.shape)

(2238643, 69)   (559661, 69)
(2238643, 7)   (559661, 7)


In [ ]:
#selecting 1% of random rows for better running time

X_train = X_train.sample(frac=0.1, replace=True, random_state=1)
y_train = y_train.sample(frac=0.1, replace=True, random_state=1)
X_test = X_test.sample(frac=0.1, replace=True, random_state=1)
y_test = y_test.sample(frac=0.1, replace=True, random_state=1)
print (X_train.shape, y_train.shape)
print( X_test.shape, y_test.shape)

(223864, 69) (223864, 7)
(55966, 69) (55966, 7)


In [ ]:
X_train=np.array(X_train)
X_test=np.array(X_test)
y=np.array(y)

In [ ]:
X_train = np.reshape(X_train, ( X_train.shape[0], 1 , X_train.shape[1] ))
X_test = np.reshape(X_test, ( X_test.shape[0], 1,  X_test.shape[1] ))

In [ ]:
from keras.layers import Dense, LSTM, MaxPool1D, Flatten, Dropout # importing dense layer
from keras.models import Sequential #importing Sequential layer
from keras.layers import Input
from keras.models import Model

model = Sequential() # initializing model
model.add(LSTM(64,return_sequences=True,input_shape = (1, X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(64,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64,return_sequences=True))
model.add(Flatten())
model.add(Dense(units=50))
# output layer with softmax activation
model.add(Dense(units=7,activation='softmax'))

In [ ]:
# defining loss function, optimizer, metrics and then compiling model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# summary of model layers
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 64)             34304     
                                                                 
 dropout (Dropout)           (None, 1, 64)             0         
                                                                 
 lstm_1 (LSTM)               (None, 1, 64)             33024     
                                                                 
 dropout_1 (Dropout)         (None, 1, 64)             0         
                                                                 
 lstm_2 (LSTM)               (None, 1, 64)             33024     
                                                                 
 flatten (Flatten)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 50)                3

In [ ]:
history = model.fit(X_train, y_train, epochs=100, batch_size=5000,validation_split=0.2)

Epoch 1/100


36/36 [==============================] - 8s 86ms/step - loss: 1.4773 - accuracy: 0.9070 - val_loss: 0.5155 - val_accuracy: 0.9263
Epoch 2/100
36/36 [==============================] - 2s 54ms/step - loss: 0.4187 - accuracy: 0.9242 - val_loss: 0.3494 - val_accuracy: 0.9263
Epoch 3/100
36/36 [==============================] - 2s 56ms/step - loss: 0.3344 - accuracy: 0.9242 - val_loss: 0.3089 - val_accuracy: 0.9263
Epoch 4/100
36/36 [==============================] - 2s 58ms/step - loss: 0.3090 - accuracy: 0.9242 - val_loss: 0.2930 - val_accuracy: 0.9263
Epoch 5/100
36/36 [==============================] - 2s 58ms/step - loss: 0.2948 - accuracy: 0.9242 - val_loss: 0.2810 - val_accuracy: 0.9263
Epoch 6/100
36/36 [==============================] - 3s 79ms/step - loss: 0.2829 - accuracy: 0.9242 - val_loss: 0.2696 - val_accuracy: 0.9262
Epoch 7/100
36/36 [==============================] - 3s 82ms/step - loss: 0.2715 - accuracy: 0.9241 - val_loss: 0.2581 - val_accuracy: 0.9262
Epoc

In [ ]:
# predicting target attribute on testing dataset
test_results = model.evaluate(X_test, y_test, verbose=1)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]*100}%')

1749/1749 [==============================] - 3s 2ms/step - loss: 0.1534 - accuracy: 0.9586
Test results - Loss: 0.15335828065872192 - Accuracy: 95.85819840431213%
